In [ ]:
file_path = 'FinalVersionV2.csv'
data = pd.read_csv(file_path, encoding='ISO-8859-1')
# Define colors based on sentiment
# For Trump (red tones) and Clinton (blue tones) with different gray levels for sentiments
sentiment_colors_trump = {
    'positive': (1, 0, 0, 0.6),  # Medium red
    'negative': (1, 0, 0, 0.9),  # Dark red
    'neutral': (1, 0, 0, 0.3)    # Light red
}

sentiment_colors_clinton = {
    'positive': (0, 0, 1, 0.6),  # Medium blue
    'negative': (0, 0, 1, 0.9),  # Dark blue
    'neutral': (0, 0, 1, 0.3)    # Light blue
}

# Get frequencies for each sentiment-tone combination for both Trump and Clinton
sentiment_freq_trump = {
    sentiment: data_trump[data_trump['semtiment'] == sentiment]['tone'].value_counts().reindex(tones, fill_value=0).values
    for sentiment in sentiments
}

sentiment_freq_clinton = {
    sentiment: data_clinton[data_clinton['semtiment'] == sentiment]['tone'].value_counts().reindex(tones, fill_value=0).values
    for sentiment in sentiments
}

# Append first element to the end for closing the radar plot
for sentiment in sentiments:
    sentiment_freq_trump[sentiment] = np.append(sentiment_freq_trump[sentiment], sentiment_freq_trump[sentiment][0])
    sentiment_freq_clinton[sentiment] = np.append(sentiment_freq_clinton[sentiment], sentiment_freq_clinton[sentiment][0])

# Plot radar charts side by side with different gray levels for each sentiment
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7), subplot_kw=dict(polar=True))
fig.suptitle("Tone and Sentiment Radar: Trump (Red) vs. Clinton (Blue)", fontsize=16)

# Trump Radar Plot
ax1.set_theta_offset(np.pi / 2)
ax1.set_theta_direction(-1)
ax1.set_xticks(angles[:-1])
ax1.set_xticklabels(tones)
for sentiment, color in sentiment_colors_trump.items():
    ax1.plot(angles, sentiment_freq_trump[sentiment], color=color, linewidth=1, linestyle='solid')
    ax1.fill(angles, sentiment_freq_trump[sentiment], color=color, alpha=0.25)
ax1.set_title("Trump")

# Clinton Radar Plot
ax2.set_theta_offset(np.pi / 2)
ax2.set_theta_direction(-1)
ax2.set_xticks(angles[:-1])
ax2.set_xticklabels(tones)
for sentiment, color in sentiment_colors_clinton.items():
    ax2.plot(angles, sentiment_freq_clinton[sentiment], color=color, linewidth=1, linestyle='solid')
    ax2.fill(angles, sentiment_freq_clinton[sentiment], color=color, alpha=0.25)
ax2.set_title("Clinton")

# Display the plots
plt.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go



# 提取节点信息
topics = data['topics'].unique().tolist()
handles = ['realDonaldTrump', 'HillaryClinton']
sentiments = data['semtiment'].unique().tolist()

# 定义所有节点和位置
all_nodes = topics + handles + sentiments
node_positions = {name: idx for idx, name in enumerate(all_nodes)}

# 创建链接（topic -> handle 和 handle -> sentiment）
links = []
for topic in topics:
    for handle in handles:
        count = len(data[(data['topics'] == topic) & (data['handle'] == handle)])
        if count > 0:
            links.append({
                'source': node_positions[topic],
                'target': node_positions[handle],
                'value': count,
                'color': '#A9A9A9' if handle == 'realDonaldTrump' else '#D3D3D3'  # 深灰和浅灰
            })

for handle in handles:
    for sentiment in sentiments:
        count = len(data[(data['handle'] == handle) & (data['semtiment'] == sentiment)])
        if count > 0:
            links.append({
                'source': node_positions[handle],
                'target': node_positions[sentiment],
                'value': count,
                'color': '#A9A9A9' if handle == 'realDonaldTrump' else '#D3D3D3'  # 深灰和浅灰
            })

# 设置节点颜色
node_colors = ['#A9A9A9' if node in handles else '#D3D3D3' for node in all_nodes]  # 深灰和浅灰

# 创建桑葚图
sankey_data = go.Sankey(
    node=dict(
        pad=10,             # 简单的填充
        thickness=15,       # 较薄的节点
        line=dict(color="gray", width=0.5),  # 细边框
        label=all_nodes,
        color=node_colors
    ),
    link=dict(
        source=[link['source'] for link in links],
        target=[link['target'] for link in links],
        value=[link['value'] for link in links],
        color=[link['color'] for link in links]  # 两种灰色的链接颜色
    )
)

# 创建并显示简化的桑葚图
fig = go.Figure(data=[sankey_data])
fig.update_layout(
    title_text="Sankey Diagram: Topics -> Handle -> Sentiment",  # 简单标题
    font_size=10,  # 适中字体大小
    margin=dict(l=20, r=20, t=20, b=20)  # 较小边距
)
fig.show()